In [8]:
import os

INPUT_DIR = "../data/nat-ai/text/"
filenames = os.listdir(INPUT_DIR)

texts = []  # list of documents
for filename in filenames:
    with open(INPUT_DIR+filename) as f:
        texts.append(f.read())  # read the whole document

len(texts)

19

In [9]:
from stopwordsiso import stopwords

stopwords = stopwords("en")

len(stopwords)

1298

In [10]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/arthit/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/arthit/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/arthit/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer

class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if (t.isalpha() and len(t) >= 2)]

bow_vectorizer = CountVectorizer(stop_words=stopwords, tokenizer=LemmaTokenizer(), lowercase=True, min_df=10)
bow_matrix = bow_vectorizer.fit_transform(texts)

bow_matrix.shape

(19, 794)

In [12]:
from sklearn.decomposition import LatentDirichletAllocation as LDA

lda_bow  = LDA(n_components=5, random_state=42) 
lda_bow.fit(bow_matrix)

LatentDirichletAllocation(n_components=5, random_state=42)

In [13]:
lda_bow.transform(bow_matrix[:2])

array([[7.24297968e-01, 2.53667830e-01, 1.07646825e-03, 8.08031995e-05,
        2.08769313e-02],
       [9.57086414e-05, 3.77953483e-01, 3.06606209e-01, 9.56388052e-05,
        3.15248961e-01]])

In [14]:
for idx, topic in enumerate(lda_bow.components_):
    print(f"Top 5 words in Topic #{idx}:")
    print([bow_vectorizer.get_feature_names()[i] for i in topic.argsort()[-5:]]) 
    print('')

Top 5 words in Topic #0:
['digital', 'public', 'measure', 'development', 'data']

Top 5 words in Topic #1:
['ha', 'technology', 'development', 'public', 'data']

Top 5 words in Topic #2:
['european', 'support', 'strategy', 'development', 'digital']

Top 5 words in Topic #3:
['ministry', 'action', 'application', 'data', 'government']

Top 5 words in Topic #4:
['business', 'ha', 'data', 'intelligence', 'artificial']



In [15]:
import pyLDAvis 
import pyLDAvis.sklearn 

pyLDAvis.enable_notebook()

/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:13: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _np_version_under1p19 = _nlv < LooseVersion("1.19")
/usr/local/lib/python3.9/site-packages/pandas/compat/numpy/__init__.py:14: DeprecationWarning: distutils Version classes are deprecat

In [17]:
pyLDAvis.sklearn.prepare(lda_bow, bow_matrix, bow_vectorizer)

/usr/local/lib/python3.9/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.026490 -0.008783       1        1  31.361702
2      0.049149 -0.015593       2        1  21.891269
0      0.085742  0.024856       3        1  17.344730
4     -0.064607  0.077773       4        1  15.779638
3     -0.043795 -0.078253       5        1  13.622662, topic_info=             Term         Freq        Total Category  logprob  loglift
55     artificial  1847.000000  1847.000000  Default  30.0000  30.0000
405  intelligence  1837.000000  1837.000000  Default  29.0000  29.0000
344    government   913.000000   913.000000  Default  28.0000  28.0000
483       measure   623.000000   623.000000  Default  27.0000  27.0000
193          data  2574.000000  2574.000000  Default  26.0000  26.0000
..            ...          ...          ...      ...      ...      ...
502      national   104.468297   740.445196   Topic5  -4.9150   0.0351
405  intelligence   123.167535  1837.613908   Topic5  -4.7503  -0.7092
55     artificial   109.066410  1847.867009   Topic5  -4.8719  -0.8364
94       business    93.330354   779.654393   Topic5  -5.0277  -0.1293
665        sector    92.656212  1002.494201   Topic5  -5.0349  -0.3879

[345 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1         1  0.399719  academia
1         2  0.559607  academia
1         3  0.031978  academia
6         1  0.435531    access
6         2  0.080654    access
...     ...       ...       ...
789       1  0.428199        wa
789       2  0.084699        wa
789       3  0.028233        wa
789       4  0.414082        wa
789       5  0.042349        wa

[1080 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 1, 5, 4])